## Load Data

In [ ]:
import pandas as pd

# Load the dataframes from csv files
transactions = pd.read_csv("https://storage.googleapis.com/datalynn-datasets/interview-challenge/technology/Amazon_AWS/transactions.csv")
customer_interactions = pd.read_csv("https://storage.googleapis.com/datalynn-datasets/interview-challenge/technology/Amazon_AWS/customer_interactions.csv")
customer_details = pd.read_csv("https://storage.googleapis.com/datalynn-datasets/interview-challenge/technology/Amazon_AWS/customer_details.csv")

## Question1
**Original Question**: Write an SQL query to find the probability of purchase for each product category in the `Transactions` table given the conditions in the `Customer_Interactions` table, considering that customers are more likely to purchase a product if they have interacted with it more than 3 times in the past month. Explain how you use Bayes theorem in this context.

**Problem Solving Ideas and Tips**:
We can compute the conditional probability of a purchase given more than 3 interactions with a product in the past month. Using Bayes' theorem, we can express this as:

$$
 P(\text{Purchase} | \text{Interactions} > 3) = \frac{P(\text{Interactions} > 3 | \text{Purchase}) \times P(\text{Purchase})}{P(\text{Interactions} > 3)}
$$

Where:
- $ P(\text{Purchase} | \text{Interactions} > 3) $: Probability of purchase given more than 3 interactions.
- $ P(\text{Interactions} > 3 | \text{Purchase}) $: Probability of having more than 3 interactions given a purchase.
- $ P(\text{Purchase}) $: Overall probability of a purchase.
- $ P(\text{Interactions} > 3) $: Overall probability of more than 3 interactions.

### Solution:


In [ ]:
import sqlite3
import pandas as pd

# Convert the 'interaction_date' column to datetime
customer_interactions['interaction_date'] = pd.to_datetime(customer_interactions['interaction_date'])

# Define a date 30 days ago from the latest interaction date in the dataset
latest_date = customer_interactions['interaction_date'].max()
date_30_days_ago = latest_date - pd.Timedelta(days=30)

# Create an SQLite3 connection
conn = sqlite3.connect('example.db')

# Write dataframes to tables
transactions.to_sql('Transactions', conn, if_exists='replace', index=False)
customer_interactions.to_sql('Customer_Interactions', conn, if_exists='replace', index=False)

# Now you can execute the original SQLite query
query = f'''
WITH Interactions_CTE AS (
    SELECT product_category, customer_id, COUNT(interaction_id) AS interaction_count
    FROM Customer_Interactions
    WHERE interaction_date > "{date_30_days_ago}"
    GROUP BY product_category, customer_id
    HAVING interaction_count > 3
),
Purchases_CTE AS (
    SELECT product_category, COUNT(transaction_id) AS purchase_count
    FROM Transactions
    GROUP BY product_category
),
Interactions_Given_Purchase_CTE AS (
    SELECT i.product_category, COUNT(DISTINCT i.customer_id) AS interactions_given_purchase_count
    FROM Interactions_CTE i
    JOIN Transactions t ON i.product_category = t.product_category AND i.customer_id = t.customer_id
    GROUP BY i.product_category
)

SELECT
    t.product_category,
    COALESCE((CAST(interactions_given_purchase_count AS FLOAT) * CAST(purchase_count AS FLOAT)) /
        (CAST(interaction_count AS FLOAT) * CAST(purchase_count AS FLOAT)), 0) AS probability_of_purchase
FROM Purchases_CTE t
LEFT JOIN (SELECT product_category, SUM(interaction_count) AS interaction_count FROM Interactions_CTE GROUP BY product_category) i
ON i.product_category = t.product_category
LEFT JOIN Interactions_Given_Purchase_CTE igp ON igp.product_category = t.product_category;
'''

# Execute the corrected query and fetch the results
result = pd.read_sql_query(query, conn)

result


,product_category,probability_of_purchase
0,books,0.233115
1,clothing,0.000000
2,electronics,0.000000
3,groceries,0.000000
4,home_appliances,0.000000
5,sports,0.000000
6,toys,0.000000


## Question2
**Original Question**: Assume we want to predict if a customer is likely to make a purchase in the next seven days based on their interaction data. Develop a supervised machine learning model using the features you think would be most relevant from the `Customer_Interactions` and `Customer_Details` tables. Explain your feature selection, the machine learning algorithm you chose, and why.

**Problem Solving Ideas and Tips**:
Predicting if a customer is likely to make a purchase in the next seven days is a binary classification problem. The goal is to utilize the historical interaction data and customer details to build a predictive model.

**Feature Selection**:
1. **From Customer_Interactions**:
   - `interaction_type`: Different interaction types like view, click, or add_to_cart might have different correlations with the likelihood of purchase.
   - `interaction_date`: The recent interactions are likely to have a higher correlation with purchasing in the near future.
   - `product_category`: The categories of products interacted with can also be indicative of interests.

2. **From Customer_Details**:
   - `gender`, `age`, `location`, `profession`: These demographic details may have correlations with purchasing preferences and likelihood.

**Model Selection**:
A Logistic Regression model is a good starting point for binary classification problems, as it is interpretable and gives the relationship between the features and the probability of the positive class. It can handle both numerical and categorical features, and its simplicity makes it a good first model to try.

**Implementation Steps**:
1. **Data Preparation**: Merge the `Customer_Interactions` and `Customer_Details` tables on `customer_id`. Encode categorical variables (e.g., `interaction_type`, `gender`, `location`, `profession`) using techniques like one-hot encoding. Consider time-based features from the `interaction_date`, such as days since the last interaction.
2. **Target Variable Definition**: Determine whether the customer has made a purchase in the next seven days. This will be the target variable.
3. **Model Training**: Split the dataset into training and test sets, and then train the Logistic Regression model using the training set.
4. **Model Evaluation**: Evaluate the model using metrics like accuracy, precision, recall, F1-score, and ROC AUC on the test set.
5. **Model Interpretation**: Interpret the model by analyzing the coefficients of the features to understand their effects.

### Solution:


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Merge Customer_Interactions and Customer_Details on customer_id
merged_data = customer_interactions_df.merge(customer_details_df, on='customer_id', how='left')

# Define the target variable
# For each interaction, check if the customer made a purchase within the next 7 days
merged_data['next_7_days'] = merged_data['interaction_date'] + pd.Timedelta(days=7)
merged_data['purchased_in_next_7_days'] = merged_data.apply(
    lambda row: transactions_df[(transactions_df['customer_id'] == row['customer_id']) &
                                (transactions_df['purchase_date'] <= row['next_7_days']) &
                                (transactions_df['purchase_date'] > row['interaction_date'])].shape[0] > 0, axis=1
).astype(int)

# Drop the temporary column
merged_data.drop('next_7_days', axis=1, inplace=True)

# Encode categorical variables using one-hot encoding
encoded_data = pd.get_dummies(merged_data, columns=['interaction_type', 'gender', 'location', 'profession', 'product_category'])

# Define feature_columns
feature_columns = encoded_data.columns.tolist()
feature_columns.remove('purchased_in_next_7_days')
feature_columns.remove('interaction_id')
feature_columns.remove('interaction_date')
feature_columns.remove('customer_id')

# Extract features and target variable
X = encoded_data[feature_columns]
y = encoded_data['purchased_in_next_7_days']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print('Accuracy:', accuracy)


Accuracy: 0.932


## Question3

### Original Question:
Develop a time series model to forecast the transaction amount for each product category in the next month. Also, provide a confidence interval for your forecasts. Explain your model selection and how you evaluated its performance.

### Problem Solving Ideas and Tips:
1. **Data Preparation**: We'll first need to aggregate the data by product category and by time (e.g., daily, weekly, monthly) to get the time series for each category.
2. **Seasonality Detection**: Investigate the seasonality pattern in the data, and depending on its nature, select an appropriate time series model.
3. **Model Selection**: Since there's a strong seasonal trend, a model like SARIMA (Seasonal Autoregressive Integrated Moving Average) can be a good choice. It's widely used to analyze and forecast seasonal data.
4. **Model Evaluation**: The model can be evaluated using metrics like MAE, RMSE, or MAPE, and by comparing the forecast with a validation set.
5. **Confidence Intervals**: Most time series models, including SARIMA, can provide confidence intervals for forecasts, allowing us to quantify the uncertainty in our predictions.

### Solution:





**Data Preparation**

In [ ]:
# Assuming the transactions data is in a pandas DataFrame called transactions
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])
transactions_by_category = transactions.groupby(['product_category', transactions['purchase_date'].dt.to_period("M")])['purchase_amount'].sum().reset_index()
transactions_by_category['purchase_date'] = transactions_by_category['purchase_date'].dt.to_timestamp()

**Modeling for Each Category**

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Define SARIMA parameters
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 12)  # Assuming a yearly seasonality pattern with monthly data
confidence_level = 0.95

forecasts = {}
for category in transactions_by_category['product_category'].unique():
    category_data = transactions_by_category[transactions_by_category['product_category'] == category].set_index('purchase_date')
    model = SARIMAX(category_data['purchase_amount'], order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
    fit_model = model.fit(disp=False)
    results = fit_model.get_forecast(steps=1, alpha=1-confidence_level)

    forecast = results.predicted_mean.iloc[0]
    conf_int = results.conf_int().iloc[0].tolist()

    forecasts[category] = {
        'Forecast': forecast,
        'Confidence Interval': conf_int
    }

forecasts


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No freq

{'books': {'Forecast': 1439710.6121097645,
  'Confidence Interval': [663831.9812628833, 2215589.242956646]},
 'clothing': {'Forecast': 10910.203061588829,
  'Confidence Interval': [8701.612957281717, 13118.79316589594]},
 'electronics': {'Forecast': 4588.549195918007,
  'Confidence Interval': [-3040.1439826449787, 12217.242374480993]},
 'groceries': {'Forecast': 6584.116212946939,
  'Confidence Interval': [-3185.4848229789623, 16353.717248872841]},
 'home_appliances': {'Forecast': 13270.753829758993,
  'Confidence Interval': [6800.6426806412, 19740.864978876787]},
 'sports': {'Forecast': -4441.280717474525,
  'Confidence Interval': [-19057.762964760204, 10175.201529811155]},
 'toys': {'Forecast': 3679.7065211351755,
  'Confidence Interval': [-2220.4379248438436, 9579.850967114195]}}

**Performance Evaluation**
You can split the data into training and validation sets, train the model on the training set, and evaluate it on the validation set using metrics like MAE, RMSE, or MAPE. This process can help you tune hyperparameters and select the best model.

### Real-World Extreme Situations:
1. **Missing Data**: In real-world scenarios, there might be missing data for some time periods. Proper imputation techniques or model handling of missing data might be required.
2. **Outliers**: Extreme values could skew the forecasts, so outlier detection and treatment might be necessary.
3. **Changes in Seasonality Pattern**: Sometimes, the seasonal pattern may change over time. Regularly updating and monitoring the model will be essential to capture any such changes.
4. **Short Time Series**: If there is not enough data to detect seasonality or train the model, alternative methods like exponential smoothing or simpler models might be more appropriate.

By following this approach, you can create a forecast for each product category and quantify the uncertainty in these forecasts with confidence intervals. Regularly updating the model with new data will help in maintaining the accuracy of the forecasts.

## Question4
### Original Question:
Develop a recommendation system to suggest products to customers based on their past interactions and purchases. The system should also consider the demographic details of the customers. Implement your model using Python and explain how you validated the results. Provide the code to showcase the top 5 recommendations for a customer.

### Problem Solving Ideas and Tips:
1. **Data Preparation**: The data will be merged from `Transactions` and `Customer_Interactions` tables along with `Customer_Details` to include demographic information.
2. **Model Selection**: A collaborative filtering approach with matrix factorization techniques like Singular Value Decomposition (SVD) could be appropriate. We can include the demographic information as features.
3. **Validation**: We can validate the model using metrics like precision@k, recall@k, and F1-score. A common approach is to divide the data into training and test sets and validate the recommendations on the test set.
4. **Top 5 Recommendations**: We'll create Python code to fetch the top 5 recommendations for a specific customer after storing the recommendations in a table.

### Solution:


**Data Preparation**

In [ ]:
import pandas as pd

# Merging transactions and customer_interactions
all_data = pd.merge(transactions, customer_interactions, on=['customer_id'], suffixes=('', '_from_interactions'), how='outer')

# Merging with customer_details
all_data = pd.merge(all_data, customer_details, on='customer_id', how='left')


**Model Implementation**

We can use the Surprise library for implementing SVD with additional demographic features.

In [ ]:
!pip install surprise

In [ ]:
from surprise import SVD, Dataset, Reader

# Considering only relevant columns
data_subset = all_data[['customer_id', 'product_category', 'purchase_amount', 'gender', 'age', 'location', 'profession']]

# Removing rows with NaN values (this step is necessary for the Surprise library to work correctly)
data_subset = data_subset.dropna()

# Converting to Surprise Dataset
reader = Reader(rating_scale=(data_subset['purchase_amount'].min(), data_subset['purchase_amount'].max()))
data = Dataset.load_from_df(data_subset[['customer_id', 'product_category', 'purchase_amount']], reader)

# Training the model
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

# Making predictions for a specific user
user_id = 1234
recommendations = []
for product in all_data['product_category'].unique():
    score = model.predict(user_id, product).est
    recommendations.append((product, score))

top_5_recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[:5]
top_5_recommendations

[('electronics', 999.85),
 ('clothing', 999.85),
 ('sports', 999.85),
 ('toys', 999.85),
 ('books', 999.85)]

**Validation**

Let's define a basic validation for the recommendation system using precision@k.

### Precision@k
Precision@k measures the proportion of recommended items in the top-k set that are relevant. For simplicity, let's assume that if a product category is in a customer's transaction history, then it's relevant to them.

Given this, the precision@k for our basic recommendation system can be defined as:

$$
\text{Precision@k} = \frac{\text{Number of relevant items in top-k recommendations}}{k}
$$

To compute this for our basic recommendation system:

1. We'll fetch the top-k product categories that a user has interacted with the most (our recommendations).
2. We'll fetch the product categories the user has actually purchased from.
3. We'll compute the precision@k using the formula above.

Let's write the code for this:




In [ ]:
def precision_at_k(customer_id, k, customer_interactions_df, transactions_df):
    # Get top-k product categories that the user has interacted with
    recommended_products_df = customer_interactions_df[customer_interactions_df['customer_id'] == customer_id]
    recommended_products = set(recommended_products_df['product_category'].value_counts().nlargest(k).index)

    # Get the product categories that the user has actually purchased from
    actual_products = set(transactions_df[transactions_df['customer_id'] == customer_id]['product_category'].unique())

    # Compute precision@k
    relevant_and_recommended = recommended_products.intersection(actual_products)
    precision_k = len(relevant_and_recommended) / k

    return precision_k


customer_id_test = 1234
k = 5
print(f"Precision@{k} for customer {customer_id_test}: {precision_at_k(customer_id_test, k, customer_interactions_df, transactions_df)}")


Precision@5 for customer 1234: 0.4


This code will compute the precision@k for a specific customer. You can iterate over multiple customers to get an average precision@k for a more holistic measure of your recommendation system's performance.

Note: This is a basic way to evaluate the recommendation system. In a real-world scenario, you might have more sophisticated methods like A/B testing to truly gauge the effectiveness of your recommendations.

### Real-World Extreme Situations:
1. **Scalability**: If the dataset is extremely large, the model needs to be optimized for scalability.
2. **Cold Start Problem**: For new users or products, there might be no previous interactions. You may need to handle these cases separately, possibly by recommending popular items.
3. **Diverse Demographics**: If there is a wide variety of demographic data, proper feature engineering and selection will be critical.

This recommendation system integrates both collaborative filtering and content-based approaches, leveraging past interactions and demographic details to make personalized recommendations. It would require regular updates with new data and continuous monitoring to ensure its effectiveness.

# Left blank